In [ ]:
!pip install tensorflow opencv-python numpy matplotlib tqdm

In [ ]:
import os

# Define the folder path and the file path
new_folder = "/kaggle/working/3DCAE"
# new_file_path = os.path.join(new_folder, "vid2array.txt")

# Create the new folder. The exist_ok=True flag prevents errors if it already exists.
os.makedirs(new_folder, exist_ok=True)
print(f"Directory '{new_folder}' created successfully (or already exists).")

In [ ]:
%%writefile /kaggle/working/3DCAE/vid2array.py
import os
import random

# --- Configuration for both datasets ---

# Avenue Dataset Paths
AVENUE_TRAIN_VIDEOS = "/kaggle/input/avenuedataset/Avenuedataset/Avenue_Dataset/Avenue Dataset/training_videos"
AVENUE_TEST_VIDEOS = "/kaggle/input/avenuedataset/Avenuedataset/Avenue_Dataset/Avenue Dataset/testing_videos"

# FUTD (FUTMINNA) Dataset Paths
FUTD_TRAIN_VIDEOS = "/kaggle/input/futminna-dataset/training_videos/training_videos"
FUTD_TEST_VIDEOS = "/kaggle/input/futminna-dataset/testing_videos/testing_videos"

ARMY_VIDEOS = "/kaggle/input/armybandit-dataset"
BANDIT_TEST_VIDEOS = "/kaggle/input/bandit-dataset"

# Output Paths for Combined Datasets
OUTPUT_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)
TRAINING_VIDEO_PATHS_FILE = os.path.join(OUTPUT_DIR, "combined_training_video_paths.txt")
TESTING_VIDEO_PATHS_FILE = os.path.join(OUTPUT_DIR, "combined_testing_video_paths.txt")
VALIDATION_VIDEO_PATHS_FILE = os.path.join(OUTPUT_DIR, "combined_validation_video_paths.txt")

# --- Helper Function to Get Video Paths ---
def get_video_paths_from_dir(base_dir):
    """
    Scans a directory and returns a list of video file paths.
    """
    video_paths = []
    # Check if the directory exists and is not empty
    if not os.path.exists(base_dir):
        print(f"Warning: Directory not found: {base_dir}")
        return []

    # Get all video files (assuming mp4, avi, etc.)
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_paths.append(os.path.join(root, file))
    return video_paths

# --- Main Script to Generate Combined Path Files ---
def generate_combined_paths():
    """
    Combines video paths from multiple datasets and splits them into
    training, validation, and testing sets.
    """
    print("Combining Avenue and FUTD training video paths...")
    avenue_train_paths = get_video_paths_from_dir(AVENUE_TRAIN_VIDEOS)
    futd_train_paths = get_video_paths_from_dir(FUTD_TRAIN_VIDEOS)
    army_train_paths = get_video_paths_from_dir(ARMY_VIDEOS)
    
    # Simple aggregation for training
    all_training_paths = avenue_train_paths + futd_train_paths + army_train_paths
    random.shuffle(all_training_paths) # Shuffle to mix the two datasets
    
    # Split training paths into a smaller validation set (e.g., 20%)
    validation_split_index = int(len(all_training_paths) * 0.2)
    training_paths = all_training_paths[validation_split_index:]
    validation_paths = all_training_paths[:validation_split_index]
    
    print(f"Total training videos: {len(training_paths)}")
    print(f"Total validation videos: {len(validation_paths)}")

    # Combine testing paths
    print("\nCombining Avenue, army and FUTD testing video paths...")
    avenue_test_paths = get_video_paths_from_dir(AVENUE_TEST_VIDEOS)
    futd_test_paths = get_video_paths_from_dir(FUTD_TEST_VIDEOS)
    bandit_test_path = get_video_paths_from_dir(BANDIT_TEST_VIDEOS)
    
    all_testing_paths = avenue_test_paths + futd_test_paths + bandit_test_path
    print(f"Total testing videos: {len(all_testing_paths)}")

    # Write paths to files
    with open(TRAINING_VIDEO_PATHS_FILE, 'w') as f:
        for path in training_paths:
            f.write(f"{path}\n")

    with open(VALIDATION_VIDEO_PATHS_FILE, 'w') as f:
        for path in validation_paths:
            f.write(f"{path}\n")
    
    with open(TESTING_VIDEO_PATHS_FILE, 'w') as f:
        for path in all_testing_paths:
            f.write(f"{path}\n")

    print(f"\nSuccessfully generated combined path files in {OUTPUT_DIR}:")
    print(f"- {os.path.basename(TRAINING_VIDEO_PATHS_FILE)}")
    print(f"- {os.path.basename(VALIDATION_VIDEO_PATHS_FILE)}")
    print(f"- {os.path.basename(TESTING_VIDEO_PATHS_FILE)}")

if __name__ == '__main__':
    generate_combined_paths()

In [ ]:
!python /kaggle/working/3DCAE/vid2array.py

In [ ]:
%%writefile /kaggle/working/3DCAE/data_generator.py
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import os

class VideoSequenceGenerator(Sequence):
    def __init__(self, video_paths_file, sequence_length, resize_dim=(128, 128), batch_size=1, shuffle=True): # <--- BATCH_SIZE=1, RESIZE_DIM=(128,128)
        self.video_paths = self._load_video_paths(video_paths_file)
        self.sequence_length = sequence_length
        self.resize_dim = resize_dim
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end() # Call to shuffle data at the start

    def _load_video_paths(self, file_path):
        with open(file_path, 'r') as f:
            paths = [line.strip() for line in f if line.strip()]
        return paths

    def _get_video_sequences(self, video_path):
        # This function loads frames from a single video and extracts all possible sequences
        frames = []
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return []

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, self.resize_dim)
            frame = frame / 255.0 # Normalize to [0, 1]
            frames.append(frame)
        cap.release()
        
        # Extract sequences (e.g., 8 frames each)
        sequences_from_video = []
        if len(frames) >= self.sequence_length:
            for i in range(0, len(frames) - self.sequence_length + 1):
                sequence = frames[i : i + self.sequence_length]
                sequences_from_video.append(np.array(sequence, dtype=np.float32))
        
        return sequences_from_video

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / self.batch_size))


    def __getitem__(self, idx):
        batch_video_paths_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        all_sequences_in_batch = []
        for i in batch_video_paths_indices:
            video_path = self.video_paths[i]
            sequences_from_current_video = self._get_video_sequences(video_path)
            all_sequences_in_batch.extend(sequences_from_current_video)

        if not all_sequences_in_batch:
            dummy_input = np.zeros((self.batch_size, self.sequence_length, *self.resize_dim, 3), dtype=np.float32)
            return dummy_input, dummy_input

        np.random.shuffle(all_sequences_in_batch)
        final_batch_sequences = all_sequences_in_batch[:self.batch_size]
        
        if not final_batch_sequences:
            dummy_input = np.zeros((0, self.sequence_length, *self.resize_dim, 3), dtype=np.float32)
            return dummy_input, dummy_input

        batch_sequences_array = np.array(final_batch_sequences, dtype=np.float32)

        return batch_sequences_array, batch_sequences_array

    def on_epoch_end(self):
        self.indices = np.arange(len(self.video_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indices)


In [ ]:
%%writefile /kaggle/working/3DCAE/ave_gt_conv.py
# Cell for .mat to .npy conversion (run this!)
# Cell for .mat to .npy conversion (run this!)
import os
import scipy.io
import numpy as np
from tqdm.notebook import tqdm

print("Starting conversion of .mat ground truth files to .npy...")

INPUT_GT_DIR = "/kaggle/input/avenuedataset/Avenuedataset/ground_truth_demo/ground_truth_demo/testing_label_mask"
OUTPUT_PROCESSED_DATA_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data"
OUTPUT_GT_DIR = os.path.join(OUTPUT_PROCESSED_DATA_DIR, "combined_testing_label_mask_npy")
os.makedirs(OUTPUT_GT_DIR, exist_ok=True)

mat_files = [f for f in os.listdir(INPUT_GT_DIR) if f.endswith(".mat")]

if not mat_files:
    print(f"No .mat files found in {INPUT_GT_DIR}. Please check the ground truth path.")
else:
    for mat_file in tqdm(mat_files, desc="Converting .mat to .npy"):
        mat_path = os.path.join(INPUT_GT_DIR, mat_file)
        
        try:
            mat_data = scipy.io.loadmat(mat_path)
            
            if 'volLabel' in mat_data: 
                mask_data_raw = mat_data['volLabel'] 
                
                frame_level_anomaly = None
                
                if mask_data_raw.ndim == 2 and mask_data_raw.shape[0] == 1 and mask_data_raw.dtype == object:
                    list_of_frame_masks = mask_data_raw[0]
                    frame_level_anomaly = np.array([1 if np.any(frame_mask > 0) else 0 for frame_mask in list_of_frame_masks]).astype(int)
                elif mask_data_raw.ndim == 1:
                    frame_level_anomaly = (mask_data_raw.astype(float) > 0).astype(int)
                elif mask_data_raw.ndim == 3:
                    frame_level_anomaly = np.array([1 if np.any(mask_data_raw[:, :, i].astype(float) > 0) else 0 for i in range(mask_data_raw.shape[2])])
                else:
                    print(f"Warning: Unexpected 'volLabel' shape or dtype in {mat_file}: {mask_data_raw.shape}, {mask_data_raw.dtype}. Skipping conversion.")
                    continue 
                    
                if frame_level_anomaly is not None:
                    # Fix: Correctly format the filename with a leading zero
                    base_name = os.path.splitext(mat_file)[0]
                    try:
                        video_id = int(base_name.split('_')[0])
                        npy_filename = f"{video_id:02d}_label.npy"
                    except (ValueError, IndexError):
                        # Fallback for unexpected naming, though it's less likely.
                        npy_filename = f"{base_name}_label.npy"
                    
                    np.save(os.path.join(OUTPUT_GT_DIR, npy_filename), frame_level_anomaly)
                else:
                    print(f"Warning: Could not process 'volLabel' for {mat_file} after shape handling. Skipping conversion.")

            else:
                print(f"Warning: 'volLabel' key not found in {mat_file}. Skipping conversion.")

        except Exception as e:
            print(f"ERROR: Failed to convert {mat_file}. Error: {e}")

print("Conversion complete.")
print(f"Converted .npy files are saved to: {OUTPUT_GT_DIR}")


In [ ]:
%%writefile /kaggle/working/3DCAE/train.py
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, Input, MaxPooling3D, UpSampling3D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from data_generator import VideoSequenceGenerator # Import our custom generator
from datetime import datetime

# --- Configuration ---
# Match these with what was used in data_generator.py
SEQUENCE_LENGTH = 16 # <--- REDUCED SEQUENCE LENGTH (from 16 to 8)
RESIZE_DIM = (128, 128) # <--- REDUCED IMAGE DIMENSIONS (from 128x128 to 64x64)
CHANNELS = 3 # Number of color channels (RGB)
BATCH_SIZE = 2 # <--- REDUCED BATCH SIZE (from 2 to 1)

EPOCHS = 50 # Number of training epochs
LEARNING_RATE = 0.0001
MODEL_SAVE_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/model" # Directory to save trained models
MODEL_FILENAME = "3DCAE_model_to_be_used.h5" # Name for the saved model
LOG_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/logs" # Directory for TensorBoard logs

# --- Data Paths (from vid2array.py output) ---
TRAINING_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_training_video_paths.txt"
VALIDATION_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_validation_video_paths.txt"

# Ensure model save directory exists
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)


# --- Model Architecture (3D CNN Autoencoder) ---
def build_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)

    # Encoder
    x = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x) # Pool spatially, not temporally
    
    x = Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x) # Pool temporally and spatially

    x = Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
    
    encoded = Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu', padding='same')(x) # Bottleneck

    # Decoder
    x = Conv3DTranspose(filters=256, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling3D(size=(2, 2, 2))(x)

    x = Conv3DTranspose(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = UpSampling3D(size=(2, 2, 2))(x)

    x = Conv3DTranspose(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = UpSampling3D(size=(1, 2, 2))(x) # Upsample spatially, not temporally in first dim

    decoded = Conv3DTranspose(filters=CHANNELS, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(x) # Output layer

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    return autoencoder

# --- Main Training Logic ---
if __name__ == '__main__':
    print("Setting up model and data generator...")
    input_shape = (SEQUENCE_LENGTH, RESIZE_DIM[0], RESIZE_DIM[1], CHANNELS)
    model = build_autoencoder(input_shape)
    
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mse') # Mean Squared Error for reconstruction loss

    model.summary()

    # --- Data Generators ---
    # Create the training data generator
    train_generator = VideoSequenceGenerator(
        video_paths_file=TRAINING_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    
    # Create the validation data generator
    validation_generator = VideoSequenceGenerator(
        video_paths_file=VALIDATION_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=False # Don't shuffle validation data
    )

    # Callbacks
    model_checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(MODEL_SAVE_DIR, MODEL_FILENAME),
        monitor='val_loss', # Changed monitor to validation loss
        save_best_only=True,
        save_weights_only=False,
        mode='min',
        verbose=1
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss', # Changed monitor to validation loss
        patience=5,
        restore_best_weights=True,
        mode='min',
        verbose=1
    )

    reduce_lr_callback = ReduceLROnPlateau(
        monitor='val_loss', # Changed monitor to validation loss
        factor=0.5,
        patience=2,
        min_lr=0.000001,
        mode='min',
        verbose=1
    )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, datetime.now().strftime("%Y%m%d-%H%M%S")))


    print("\nStarting model training...")
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=EPOCHS,
        callbacks=[model_checkpoint_callback, early_stopping_callback, reduce_lr_callback, tensorboard_callback],
        verbose=1
    )

    print("\nTraining finished.")


In [ ]:
%%writefile /kaggle/working/3DCAE/train_sma.py
import numpy as np
import os
import cv2
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, Conv3DTranspose, Input, MaxPooling3D, UpSampling3D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
from data_generator import VideoSequenceGenerator
from datetime import datetime

# --- Configuration ---
SEQUENCE_LENGTH = 16
RESIZE_DIM = (128, 128)
CHANNELS = 3
BATCH_SIZE = 2
EPOCHS = 50
LEARNING_RATE = 0.0001  # Default, will be optimized by SMA
MODEL_SAVE_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/model"
MODEL_FILENAME = "3DCAE_model_to_be_used.h5" 
LOG_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/logs"
TRAINING_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_training_video_paths.txt"
VALIDATION_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_validation_video_paths.txt"
TESTING_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_validation_video_paths.txt"
GROUND_TRUTH_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/ground_truth"
ANNOTATION_THRESHOLD = 0.0015

# Ensure directories exist
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# --- Ground Truth Masks ---
def load_ground_truth_masks(gt_base_dir, test_sequence_folder_path, target_resize_dim):
    masks = []
    sequence_folder_name = os.path.basename(test_sequence_folder_path)
    gt_subfolder_name = sequence_folder_name + "_gt"
    gt_folder_path = os.path.join(gt_base_dir, gt_subfolder_name)
    if not os.path.exists(gt_folder_path):
        print(f"Ground truth folder {gt_folder_path} not found, skipping...")
        return None
    mask_filenames = sorted([f for f in os.listdir(gt_folder_path) if f.endswith(('.bmp', '.png', '.jpg'))])
    for filename in mask_filenames:
        mask_path = os.path.join(gt_base_dir, gt_subfolder_name, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            continue
        mask = (mask > 0).astype(np.uint8) * 255
        if mask.shape != target_resize_dim:
            mask = cv2.resize(mask, (target_resize_dim[1], target_resize_dim[0]), interpolation=cv2.INTER_NEAREST)
        if CHANNELS == 3:  # Convert grayscale mask to RGB
            mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
        masks.append(mask)
    if not masks:
        print(f"No valid masks found in {gt_folder_path}, skipping...")
        return None
    return np.array(masks)

# --- Model Architecture ---
def build_autoencoder(input_shape, filters_1=64, kernel_size_1=3, filters_2=128, kernel_size_2=3):
    input_layer = Input(shape=input_shape)
    kernel_size_1 = (int(kernel_size_1), int(kernel_size_1), int(kernel_size_1))
    kernel_size_2 = (int(kernel_size_2), int(kernel_size_2), int(kernel_size_2))
    
    # Encoder
    x = Conv3D(filters=int(filters_1), kernel_size=kernel_size_1, activation='relu', padding='same')(input_layer)
    x = MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)
    x = Conv3D(filters=int(filters_2), kernel_size=kernel_size_2, activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
    x = Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(x)
    encoded = Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    
    # Decoder
    x = Conv3DTranspose(filters=256, kernel_size=(3, 3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling3D(size=(2, 2, 2))(x)
    x = Conv3DTranspose(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = UpSampling3D(size=(2, 2, 2))(x)
    x = Conv3DTranspose(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = UpSampling3D(size=(1, 2, 2))(x)
    decoded = Conv3DTranspose(filters=CHANNELS, kernel_size=(3, 3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    return autoencoder

# --- Objective Function for SMA ---
def objective_function(params):
    lr, filters_1, kernel_size_1, filters_2, kernel_size_2 = params
    print(f"Evaluating params: lr={lr:.6f}, filters_1={int(filters_1)}, kernel_size_1={int(kernel_size_1)}, filters_2={int(filters_2)}, kernel_size_2={int(kernel_size_2)}")
    input_shape = (SEQUENCE_LENGTH, RESIZE_DIM[0], RESIZE_DIM[1], CHANNELS)
    model = build_autoencoder(input_shape, filters_1, kernel_size_1, filters_2, kernel_size_2)
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse')
    
    train_generator = VideoSequenceGenerator(
        video_paths_file=TRAINING_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    validation_generator = VideoSequenceGenerator(
        video_paths_file=VALIDATION_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    test_generator = VideoSequenceGenerator(
        video_paths_file=TESTING_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    model_checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(MODEL_SAVE_DIR, f"temp_model_{os.getpid()}.h5"),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        mode='min',
        verbose=0
    )
    # Using patience=3 for quick evaluation within the objective function
    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        mode='min',
        verbose=0
    )
    reduce_lr_callback = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        mode='min',
        verbose=0
    )
    
    model.fit(
        train_generator,
        epochs=15, # Limited epochs for hyperparameter search
        validation_data=validation_generator,
        callbacks=[model_checkpoint_callback, early_stopping_callback, reduce_lr_callback],
        verbose=0
    )
    
    all_reconstruction_errors = []
    all_true_labels = []
    all_test_sequence_folders = test_generator._load_sequence_folder_paths(TESTING_VIDEO_PATHS_FILE)
    for sequence_folder_path in all_test_sequence_folders:
        original_frames_resized = test_generator._get_image_frames_from_folder(sequence_folder_path)
        if not original_frames_resized:
            continue
        gt_pixel_masks_for_sequence = load_ground_truth_masks(GROUND_TRUTH_DIR, sequence_folder_path, RESIZE_DIM)
        sequences_from_current_folder = []
        if len(original_frames_resized) >= SEQUENCE_LENGTH:
            for i in range(0, len(original_frames_resized) - SEQUENCE_LENGTH + 1):
                sequence = original_frames_resized[i: i + SEQUENCE_LENGTH]
                sequences_from_current_folder.append(np.array(sequence, dtype=np.float32))
        for seq_idx_in_folder, sequence in enumerate(sequences_from_current_folder):
            input_batch = np.expand_dims(sequence, axis=0)
            reconstructed_batch = model.predict(input_batch, verbose=0)
            reconstruction_error = np.mean(np.square(sequence - reconstructed_batch[0]))
            middle_frame_idx_in_sequence = SEQUENCE_LENGTH // 2
            global_frame_idx = seq_idx_in_folder + middle_frame_idx_in_sequence
            if gt_pixel_masks_for_sequence is not None and global_frame_idx < len(gt_pixel_masks_for_sequence):
                gt_segment_start = seq_idx_in_folder
                gt_segment_end = seq_idx_in_folder + SEQUENCE_LENGTH
                if gt_segment_end <= len(gt_pixel_masks_for_sequence):
                    sequence_gt_label = np.any(gt_pixel_masks_for_sequence[gt_segment_start: gt_segment_end] > 0).astype(int)
                    all_reconstruction_errors.append(reconstruction_error)
                    all_true_labels.append(sequence_gt_label)
    
    if all_reconstruction_errors and all_true_labels:
        all_reconstruction_errors = np.array(all_reconstruction_errors)
        all_true_labels = np.array(all_true_labels)
        all_true_labels = (all_true_labels > 0).astype(int)
        if len(np.unique(all_true_labels)) > 1:
            predictions = (all_reconstruction_errors > ANNOTATION_THRESHOLD).astype(int)
            f1 = f1_score(all_true_labels, predictions)
            # SMA minimizes, so return negative F1
            return -f1
    print("No valid ground truth data for F1 score, returning high loss...")
    return float('inf')

# --- Slime Mould Algorithm ---
def slime_mould_algorithm(objective_function, PopulationSize=6, MaxIters=3, dimensions=5, lb=[1e-5, 16, 3, 32, 3], ub=[1e-2, 128, 7, 256, 7], z=0.03):
    num_agents = PopulationSize
    max_iters = MaxIters
    # Set the seed for reproducibility
    np.random.seed(42) 
    random.seed(42)
    positions = np.random.uniform(lb, ub, size=(num_agents, dimensions))
    best_pos = None
    best_fitness = float('inf')

    for t in range(max_iters):
        # Calculate fitness for all positions
        # Note: This is computationally intensive as it trains a model per position per iteration
        fitness_value = np.array([objective_function(pos) for pos in positions])
        
        min_idx = np.argmin(fitness_value)
        if fitness_value[min_idx] < best_fitness:
            best_fitness = fitness_value[min_idx]
            best_pos = positions[min_idx]

        sorted_indices = np.argsort(fitness_value)
        positions = positions[sorted_indices]
        fitness_value = fitness_value[sorted_indices]
        X_b = positions[0] # Best position (leader)

        # Update weights (W) and search parameters (a, vb, vc)
        # Weight (W) calculation based on fitness difference
        # Avoid division by zero/very small number
        max_fit = np.max(fitness_value)
        min_fit = np.min(fitness_value)
        
        # Calculate W array for all agents (used in exploration/exploitation)
        # The equation for W in the original script is:
        # w = 1 + random.random() * np.log((fitness_value[0] - fitness_value[j]) / (np.max(fitness_value) - np.min(fitness_value) + 1e-10) + 1)
        # This seems to be done per agent inside the loop, so we'll keep it there.
        
        arctanh_input = np.clip(-(t / max_iters) + 1, -0.999, 0.999)
        a = np.arctanh(arctanh_input) # Global search parameter 'a'

        for j in range(num_agents):
            r = random.random()
            # Probability 'p' of transitioning from exploration to exploitation
            p = np.tanh(np.abs(fitness_value[j] - best_fitness)) 
            
            # Local search parameters
            vb = random.uniform(-a, a)
            vc = random.uniform(-(1 - t / max_iters), 1 - t / max_iters)

            # Weight calculation for the current agent
            if max_fit == min_fit:
                # Handle case where all fitness values are the same
                w = 1.0
            else:
                 # Original weight calculation from the script
                 w = 1 + random.random() * np.log((fitness_value[0] - fitness_value[j]) / (max_fit - min_fit + 1e-10) + 1)

            # Update position (Movement)
            if r < z: # Exploration (Random relocation)
                positions[j] = np.random.uniform(lb, ub, size=dimensions)
            elif r < p: # Exploitation (Movement towards best position influenced by others)
                idx_A, idx_B = random.sample(range(num_agents), 2)
                X_A, X_B = positions[idx_A], positions[idx_B]
                # Note: The original script uses X_A and X_B as randomly selected positions, 
                # which is common in SMA for simulating vein width influence.
                positions[j] = X_b + vb * (w * (X_A - X_B))
            else: # Exploitation (Local search / movement based on current position and 'vc')
                positions[j] = vc * positions[j]

            # Boundary check and constraint handling
            positions[j] = np.clip(positions[j], lb, ub)
            # Ensure odd kernel sizes for stability/consistency
            positions[j][2] = 2 * round(positions[j][2] / 2) + 1  # Ensure odd kernel_size_1
            positions[j][4] = 2 * round(positions[j][4] / 2) + 1  # Ensure odd kernel_size_2

        print(f"Iteration {t+1}/{max_iters} - Best Fitness (-F1): {best_fitness}")

    return best_pos, best_fitness

# --- Main Training Logic ---
if __name__ == '__main__':
    print("Running Slime Mould Algorithm for hyperparameter optimization...")
    lb = [1e-5, 16, 3, 32, 3]  # [lr, filters_1, kernel_size_1, filters_2, kernel_size_2]
    ub = [1e-2, 128, 7, 256, 7]
    best_params, best_fitness = slime_mould_algorithm(
        objective_function,
        PopulationSize=6,
        MaxIters=3,
        lb=lb,
        ub=ub
    )
    print(f"Optimal Hyperparameters: lr={best_params[0]:.6f}, filters_1={int(best_params[1])}, "
          f"kernel_size_1={int(best_params[2])}, filters_2={int(best_params[3])}, kernel_size_2={int(best_params[4])}")
    print(f"Best Objective (-F1): {best_fitness} (F1: {-best_fitness:.4f})")

    print("\nSetting up final model with optimized hyperparameters...")
    input_shape = (SEQUENCE_LENGTH, RESIZE_DIM[0], RESIZE_DIM[1], CHANNELS)
    model = build_autoencoder(best_params[1], best_params[2], best_params[3], best_params[4]) # Pass params to build_autoencoder
    
    optimizer = Adam(learning_rate=best_params[0])
    model.compile(optimizer=optimizer, loss='mse')
    model.summary()

    # --- Data Generators ---
    train_generator = VideoSequenceGenerator(
        video_paths_file=TRAINING_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    validation_generator = VideoSequenceGenerator(
        video_paths_file=VALIDATION_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH,
        resize_dim=RESIZE_DIM,
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    # --- Callbacks ---
    model_checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(MODEL_SAVE_DIR, MODEL_FILENAME),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        mode='min',
        verbose=1
    )
    # INTEGRATED: EarlyStopping patience = 5
    early_stopping_callback = EarlyStopping(
        monitor='val_loss',
        patience=5, 
        restore_best_weights=True,
        mode='min',
        verbose=1
    )
    # INTEGRATED: ReduceLROnPlateau min_lr = 0.000001
    reduce_lr_callback = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.000001, 
        mode='min',
        verbose=1
    )
    # Ensure TensorBoard uses tf.keras.callbacks.TensorBoard for compatibility
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOG_DIR, datetime.now().strftime("%Y%m%d-%H%M%S")))

    print("\nStarting final model training...")
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=EPOCHS,
        callbacks=[model_checkpoint_callback, early_stopping_callback, reduce_lr_callback, tensorboard_callback],
        verbose=1
    )

    print("\nTraining finished.")

In [ ]:
%%writefile /kaggle/working/3DCAE/test.py
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from data_generator import VideoSequenceGenerator
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# --- Configuration ---
# IMPORTANT: These MUST match what you successfully used for training in train.py and data_generator.py
SEQUENCE_LENGTH = 16 # <--- Matches the trained model's sequence length
RESIZE_DIM = (128, 128) # <--- Matches the trained model's resize dimensions
CHANNELS = 3
BATCH_SIZE = 2 # Use a batch size of 1 for testing to process one sequence at a time for anomaly scoring

# --- Paths ---
MODEL_PATH = "/kaggle/input/3dcae-model-project/tensorflow1/default/1/3DCAE_model_to_be_used.h5"
TESTING_VIDEO_PATHS_FILE = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_testing_video_paths.txt"
AVENUE_GROUND_TRUTH_VOL = "/kaggle/working/3DCAE/3DCAE_processed_data/combined_testing_label_mask_npy" 
RESULTS_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data/results"

os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Function to load ground truth ---
def load_ground_truth(ground_truth_path, video_path):
    video_id_str = os.path.basename(video_path).split('.')[0]
    video_id_for_gt = str(int(video_id_str)) 
    ground_truth_mask_file = os.path.join(ground_truth_path, f"{video_id_for_gt}_label.npy") 
    
    try:
        gt_labels_for_frames = np.load(ground_truth_mask_file)
        return gt_labels_for_frames
    except FileNotFoundError:
        print(f"Warning (GT File Not Found): {ground_truth_mask_file}. Video: {video_path}. Skipping for AUC.")
        return None 
    except Exception as e:
        print(f"Warning (GT Loading Error): {ground_truth_mask_file}. Error: {e}. Video: {video_path}. Skipping for AUC.")
        return None

# --- Main Evaluation Logic ---
if __name__ == '__main__':
    print("Loading trained model...")
    try:
        model = load_model(MODEL_PATH, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
        print("Model loaded successfully.")
    except Exception as e:
        print(f"ERROR: Failed to load model from {MODEL_PATH}. Error: {e}")
        print("Please ensure training completed and the model file exists.")
        exit(1)

    print("Setting up test data generator...")
    test_generator = VideoSequenceGenerator(
        video_paths_file=TESTING_VIDEO_PATHS_FILE,
        sequence_length=SEQUENCE_LENGTH, # <--- ENSURE THIS MATCHES
        resize_dim=RESIZE_DIM, # <--- ENSURE THIS MATCHES
        batch_size=BATCH_SIZE, 
        shuffle=False 
    )
    
    video_paths_for_testing = test_generator._load_video_paths(TESTING_VIDEO_PATHS_FILE)
    print(f"DEBUG TEST: Total test video paths loaded: {len(video_paths_for_testing)}")
    if len(video_paths_for_testing) > 0:
        print(f"DEBUG TEST: First test video path: {video_paths_for_testing[0]}")
    else:
        print("DEBUG TEST: No test video paths found in the list.")


    all_reconstruction_errors = []
    all_true_labels = [] 

    print("\nStarting evaluation on test videos...")
    
    if not video_paths_for_testing:
        print("ERROR: No test video paths available to evaluate. Exiting test script.")
        exit(1)

    for video_path in tqdm(video_paths_for_testing, desc="Evaluating Test Videos"):
        gt_labels_for_frames = load_ground_truth(AVENUE_GROUND_TRUTH_VOL, video_path)
        
        sequences_from_current_video = test_generator._get_video_sequences(video_path)
        
        if not sequences_from_current_video:
            continue

        video_sequence_errors = []
        for sequence in sequences_from_current_video:
            input_batch = np.expand_dims(sequence, axis=0)
            reconstructed_batch = model.predict(input_batch, verbose=0)
            
            reconstruction_error = np.mean(np.square(sequence - reconstructed_batch[0]))
            video_sequence_errors.append(reconstruction_error)

        if gt_labels_for_frames is not None and len(gt_labels_for_frames) >= SEQUENCE_LENGTH:
            gt_labels_for_sequences = []
            for i in range(len(video_sequence_errors)): 
                if (i + SEQUENCE_LENGTH) <= len(gt_labels_for_frames):
                    if np.any(gt_labels_for_frames[i : i + SEQUENCE_LENGTH] > 0):
                        gt_labels_for_sequences.append(1)
                    else:
                        gt_labels_for_sequences.append(0)
                else:
                    gt_labels_for_sequences.append(0)
            
            if len(gt_labels_for_sequences) == len(video_sequence_errors):
                all_reconstruction_errors.extend(video_sequence_errors)
                all_true_labels.extend(gt_labels_for_sequences)
            else:
                print(f"Warning: Mismatch between sequences ({len(video_sequence_errors)}) and GT labels ({len(gt_labels_for_sequences)}) for {video_path}. Skipping for AUC.")
        else:
            print(f"Warning: No valid ground truth to match sequences for {video_path}. Skipping for AUC.")

    if not all_reconstruction_errors or not all_true_labels:
        print("No valid test sequences with matching ground truth were processed. Cannot calculate AUC metrics.")
    else:
        all_reconstruction_errors = np.array(all_reconstruction_errors)
        all_true_labels = np.array(all_true_labels)
        
        all_true_labels = (all_true_labels > 0).astype(int)
        
        # --- Save results to .npy files ---
        results_save_path = os.path.join(RESULTS_DIR, "reconstruction_errors.npy")
        labels_save_path = os.path.join(RESULTS_DIR, "true_labels.npy")
        np.save(results_save_path, all_reconstruction_errors)
        np.save(labels_save_path, all_true_labels)
        print(f"Reconstruction errors saved to: {results_save_path}")
        print(f"True labels saved to: {labels_save_path}")

        if len(np.unique(all_true_labels)) > 1: 
            fpr, tpr, thresholds = roc_curve(all_true_labels, all_reconstruction_errors)
            roc_auc = auc(fpr, tpr)

            print(f"\nEvaluation Complete.")
            print(f"Total sequences processed: {len(all_reconstruction_errors)}")
            print(f"Total anomalous sequences: {np.sum(all_true_labels)}")
            print(f"Anomaly Detection AUC: {roc_auc:.4f}")

            # --- Metrics at a fixed threshold (now for reference, will be interactive in notebook) ---
            # FIXED_THRESHOLD = 0.015 # This is just a placeholder here, actual tuning done interactively
            # y_pred = (all_reconstruction_errors > FIXED_THRESHOLD).astype(int)
            # acc = accuracy_score(all_true_labels, y_pred, zero_division=0) # zero_division handles cases where precision/recall is 0
            # prec = precision_score(all_true_labels, y_pred, zero_division=0)
            # rec = recall_score(all_true_labels, y_pred, zero_division=0)
            # f1 = f1_score(all_true_labels, y_pred, zero_division=0)
            # print(f"\n(Example) Metrics at Threshold = {FIXED_THRESHOLD:.4f}:")
            # print(f"  Accuracy:  {acc:.4f}")
            # print(f"  Precision: {prec:.4f}")
            # print(f"  Recall:    {rec:.4f}")
            # print(f"  F1-Score:  {f1:.4f}")

            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.4f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic (ROC) Curve')
            plt.legend(loc="lower right")
            plt.grid(True)
            plt.savefig(os.path.join(RESULTS_DIR, "roc_curve.png"))
            plt.show()
            print(f"ROC curve saved to {os.path.join(RESULTS_DIR, 'roc_curve.png')}")
        else:
            print("Warning: Only one class (normal or anomalous) found in true labels. Cannot calculate full metrics.")
            print(f"Total sequences processed: {len(all_reconstruction_errors)}")
            print(f"Total anomalous sequences: {np.sum(all_true_labels)}")


In [ ]:
%%writefile /kaggle/working/3DCAE/video_visual.py
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import collections
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML
import shutil

# --- Configuration ---
# IMPORTANT: These MUST match your trained model
MODEL_PATH = "/kaggle/working/3DCAE/3DCAE_processed_data/model/3DCAE_model_to_be_used.h5"
SEQUENCE_LENGTH = 16
RESIZE_DIM = (128, 128)
CHANNELS = 3
# A placeholder threshold. You MUST tune this value based on your specific video data.
ANOMALY_THRESHOLD = 0.015 
# Placeholder for your video file.
VIDEO_SOURCE = "/kaggle/input/futminna-dataset/testing_videos/testing_videos/19.avi"
# New output path for the video file
OUTPUT_VIDEO_PATH = "anomaly_visualization_army_fut_testing_19.mp4"
# Limit the number of frames to process for notebook display
MAX_FRAMES_TO_PROCESS = 450

# --- Main Logic ---
def notebook_visualizer():
    """
    Simulates a live CCTV feed, detects anomalies, and displays output in a notebook.
    """
    print("Loading trained autoencoder model...")
    try:
        model = load_model(MODEL_PATH, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
        print("Model loaded successfully.")
    except Exception as e:
        print(f"ERROR: Failed to load model from {MODEL_PATH}. Please ensure it exists.")
        print(f"Error details: {e}")
        return

    print("Opening video source...")
    cap = cv2.VideoCapture(VIDEO_SOURCE)

    if not cap.isOpened():
        print(f"ERROR: Could not open video source {VIDEO_SOURCE}")
        return

    # Get video properties for the output video writer
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Use 'mp4v' for .mp4 format
    video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

    sequence_buffer = collections.deque(maxlen=SEQUENCE_LENGTH)
    frame_count = 0
    all_reconstruction_errors = []

    print(f"\nStarting anomaly visualization. Processing frames and writing to {OUTPUT_VIDEO_PATH}...")

    try:
        while frame_count < MAX_FRAMES_TO_PROCESS:
            ret, frame = cap.read()
            if not ret:
                print("End of video or read error.")
                break

            processed_frame = cv2.resize(frame, RESIZE_DIM, interpolation=cv2.INTER_AREA)
            processed_frame = processed_frame / 255.0  # Normalize to [0, 1]
            
            sequence_buffer.append(processed_frame)

            if len(sequence_buffer) == SEQUENCE_LENGTH:
                input_sequence = np.expand_dims(np.array(list(sequence_buffer)), axis=0)
                reconstructed_sequence = model.predict(input_sequence, verbose=0)
                
                original_last_frame = input_sequence[0, -1]
                reconstructed_last_frame = reconstructed_sequence[0, -1]
                
                reconstruction_error = np.mean(np.square(original_last_frame - reconstructed_last_frame))
                all_reconstruction_errors.append(reconstruction_error)
                
                display_frame = frame.copy()
                text = f"Score: {reconstruction_error:.4f}"
                
                if reconstruction_error > ANOMALY_THRESHOLD:
                    text_color = (0, 0, 255)  # Red (BGR)
                    alert_text = "ANOMALY DETECTED!"
                    cv2.putText(display_frame, alert_text, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                else:
                    text_color = (0, 255, 0)  # Green (BGR)

                cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
                
                # Write the frame to the output video file
                video_writer.write(display_frame)

            frame_count += 1
            print(f"Processed frame {frame_count}/{MAX_FRAMES_TO_PROCESS}", end='\r')

        print("\nFinished processing frames. Displaying results...")

    finally:
        cap.release()
        video_writer.release()
        print(f"Video saved to {OUTPUT_VIDEO_PATH}")

    # --- Plot reconstruction error over time ---
    plt.figure(figsize=(12, 6))
    plt.plot(all_reconstruction_errors, label='Reconstruction Error')
    plt.axhline(y=ANOMALY_THRESHOLD, color='r', linestyle='--', label=f'Anomaly Threshold ({ANOMALY_THRESHOLD})')
    plt.xlabel('Frame')
    plt.ylabel('Error')
    plt.title('Reconstruction Error Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == '__main__':
    notebook_visualizer()


In [ ]:
# Cell for .mat to .npy conversion (run this!)
import os
import scipy.io
import numpy as np
from tqdm.notebook import tqdm

print("Starting conversion of .mat ground truth files to .npy...")

INPUT_GT_DIR = "/kaggle/input/avenuedataset/Avenuedataset/ground_truth_demo/ground_truth_demo/testing_label_mask"
OUTPUT_PROCESSED_DATA_DIR = "/kaggle/working/3DCAE/3DCAE_processed_data"
OUTPUT_GT_DIR = os.path.join(OUTPUT_PROCESSED_DATA_DIR, "combined_testing_label_mask_npy")
os.makedirs(OUTPUT_GT_DIR, exist_ok=True)

mat_files = [f for f in os.listdir(INPUT_GT_DIR) if f.endswith(".mat")]

if not mat_files:
    print(f"No .mat files found in {INPUT_GT_DIR}. Please check the ground truth path.")
else:
    for mat_file in tqdm(mat_files, desc="Converting .mat to .npy"):
        mat_path = os.path.join(INPUT_GT_DIR, mat_file)
        
        try:
            mat_data = scipy.io.loadmat(mat_path)
            
            if 'volLabel' in mat_data: 
                mask_data_raw = mat_data['volLabel'] 
                
                frame_level_anomaly = None
                
                if mask_data_raw.ndim == 2 and mask_data_raw.shape[0] == 1 and mask_data_raw.dtype == object:
                    list_of_frame_masks = mask_data_raw[0]
                    frame_level_anomaly = np.array([1 if np.any(frame_mask > 0) else 0 for frame_mask in list_of_frame_masks]).astype(int)
                elif mask_data_raw.ndim == 1:
                    frame_level_anomaly = (mask_data_raw.astype(float) > 0).astype(int)
                elif mask_data_raw.ndim == 3:
                    frame_level_anomaly = np.array([1 if np.any(mask_data_raw[:, :, i].astype(float) > 0) else 0 for i in range(mask_data_raw.shape[2])])
                else:
                    print(f"Warning: Unexpected 'volLabel' shape or dtype in {mat_file}: {mask_data_raw.shape}, {mask_data_raw.dtype}. Skipping conversion.")
                    continue 
                    
                if frame_level_anomaly is not None:
                    # **FIX IS HERE: Removed the :02d format specifier**
                    base_name = os.path.splitext(mat_file)[0]
                    try:
                        # Extract the number (e.g., '04' -> 4)
                        video_id = int(base_name.split('_')[0]) 
                        # Format as non-padded string (e.g., 4 -> '4_label.npy')
                        npy_filename = f"{video_id}_label.npy" 
                    except (ValueError, IndexError):
                        # Fallback
                        npy_filename = f"{base_name}_label.npy"
                    
                    np.save(os.path.join(OUTPUT_GT_DIR, npy_filename), frame_level_anomaly)
                else:
                    print(f"Warning: Could not process 'volLabel' for {mat_file} after shape handling. Skipping conversion.")

            else:
                print(f"Warning: 'volLabel' key not found in {mat_file}. Skipping conversion.")

        except Exception as e:
            print(f"ERROR: Failed to convert {mat_file}. Error: {e}")

print("Conversion complete.")
print(f"Converted .npy files are saved to: {OUTPUT_GT_DIR}")


In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import collections
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML
import shutil

# --- Configuration ---
# IMPORTANT: These MUST match your trained model
MODEL_PATH = "/kaggle/input/3dcae-model-project/tensorflow1/default/1/3DCAE_model_to_be_used.h5"
SEQUENCE_LENGTH = 16
RESIZE_DIM = (128, 128)
CHANNELS = 3
# A placeholder threshold. You MUST tune this value based on your specific video data.
ANOMALY_THRESHOLD = 0.015 
# Placeholder for your video file.
VIDEO_SOURCE = "/kaggle/input/bandit-dataset/VID-20250917-WA0008.mp4"
# New output path for the video file
OUTPUT_VIDEO_PATH = "ave2.mp4"
# Limit the number of frames to process for notebook display
MAX_FRAMES_TO_PROCESS = 450

# --- Main Logic ---
def notebook_visualizer():
    """
    Simulates a live CCTV feed, detects anomalies, and displays output in a notebook.
    """
    print("Loading trained autoencoder model...")
    try:
        model = load_model(MODEL_PATH, custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
        print("Model loaded successfully.")
    except Exception as e:
        print(f"ERROR: Failed to load model from {MODEL_PATH}. Please ensure it exists.")
        print(f"Error details: {e}")
        return

    print("Opening video source...")
    cap = cv2.VideoCapture(VIDEO_SOURCE)

    if not cap.isOpened():
        print(f"ERROR: Could not open video source {VIDEO_SOURCE}")
        return

    # Get video properties for the output video writer
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Use 'mp4v' for .mp4 format
    video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

    sequence_buffer = collections.deque(maxlen=SEQUENCE_LENGTH)
    frame_count = 0
    all_reconstruction_errors = []

    print(f"\nStarting anomaly visualization. Processing frames and writing to {OUTPUT_VIDEO_PATH}...")

    try:
        while frame_count < MAX_FRAMES_TO_PROCESS:
            ret, frame = cap.read()
            if not ret:
                print("End of video or read error.")
                break

            processed_frame = cv2.resize(frame, RESIZE_DIM, interpolation=cv2.INTER_AREA)
            processed_frame = processed_frame / 255.0  # Normalize to [0, 1]
            
            sequence_buffer.append(processed_frame)

            if len(sequence_buffer) == SEQUENCE_LENGTH:
                input_sequence = np.expand_dims(np.array(list(sequence_buffer)), axis=0)
                reconstructed_sequence = model.predict(input_sequence, verbose=0)
                
                original_last_frame = input_sequence[0, -1]
                reconstructed_last_frame = reconstructed_sequence[0, -1]
                
                reconstruction_error = np.mean(np.square(original_last_frame - reconstructed_last_frame))
                all_reconstruction_errors.append(reconstruction_error)
                
                display_frame = frame.copy()
                text = f"Score: {reconstruction_error:.4f}"
                
                if reconstruction_error > ANOMALY_THRESHOLD:
                    text_color = (0, 0, 255)  # Red (BGR)
                    alert_text = "ANOMALY DETECTED!"
                    cv2.putText(display_frame, alert_text, (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                else:
                    text_color = (0, 255, 0)  # Green (BGR)

                cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2, cv2.LINE_AA)
                
                # Write the frame to the output video file
                video_writer.write(display_frame)

            frame_count += 1
            print(f"Processed frame {frame_count}/{MAX_FRAMES_TO_PROCESS}", end='\r')

        print("\nFinished processing frames. Displaying results...")

    finally:
        cap.release()
        video_writer.release()
        print(f"Video saved to {OUTPUT_VIDEO_PATH}")

    # --- Plot reconstruction error over time ---
    plt.figure(figsize=(12, 6))
    plt.plot(all_reconstruction_errors, label='Reconstruction Error')
    plt.axhline(y=ANOMALY_THRESHOLD, color='r', linestyle='--', label=f'Anomaly Threshold ({ANOMALY_THRESHOLD})')
    plt.xlabel('Frame')
    plt.ylabel('Error')
    plt.title('Reconstruction Error Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == '__main__':
    notebook_visualizer()


In [ ]:
!pip install tf2onnx
!pip install onnx

In [ ]:
import tensorflow as tf

# Reuse the loaded model from before (or reload if needed)
model = tf.keras.models.load_model(
    '/kaggle/input/3dcae-model-project/tensorflow1/default/1/3DCAE_model_to_be_used.h5',
    compile=False
)
model.compile(optimizer='adam', loss='mse')

print("Model ready for Flex-enabled conversion...")

# Converter with Flex (SELECT_TF_OPS) for MaxPool3D fallback
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Native ops where possible
    tf.lite.OpsSet.SELECT_TF_OPS     # Flex for tf.MaxPool3D
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantize for speed
converter.allow_custom_ops = True  # Required for Flex

try:
    tflite_model = converter.convert()
    print("✅ Flex TFLite conversion successful!")
    
    # Save
    with open('/kaggle/working/new_3DCAE_model.tflite', 'wb') as f:
        f.write(tflite_model)
    
    print(f"Model size: {len(tflite_model) / (1024*1024):.1f} MB")
    
    # Quick inference test (dummy input)
    interpreter = tf.lite.Interpreter(model_path='/kaggle/working/new_3DCAE_model.tflite')
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Dummy batch=1 input (normalize if your data needs it)
    dummy_input = tf.random.normal([1, 16, 128, 128, 3]).numpy()
    interpreter.set_tensor(input_details[0]['index'], dummy_input)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    print(f"✅ Dummy inference works! Output shape: {output.shape}, min/max: {output.min():.3f}/{output.max():.3f}")
    
except Exception as e:
    print(f"❌ Still failed: {e}")

In [ ]:
import tensorflow as tf
import numpy as np

# Test the actual saved model
interpreter = tf.lite.Interpreter(model_path='/kaggle/working/new_3DCAE_model.tflite')
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Input shape: {input_details[0]['shape']}, dtype: {input_details[0]['dtype']}")
print(f"Output shape: {output_details[0]['shape']}, dtype: {output_details[0]['dtype']}")

# Dummy batch=1 input (random normal, but scale to [0,1] if your data is images)
dummy_input = np.random.uniform(0, 1, [1, 16, 128, 128, 3]).astype(np.float32)  # Better for sigmoid output
interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

print(f"✅ Dummy inference works! Output shape: {output.shape}, min/max: {output.min():.3f}/{output.max():.3f}")
print(f"Sample MSE (anomaly score): {np.mean((dummy_input - output)**2):.6f}")